In [1]:
using Revise
using Pkg
Pkg.activate("../")
using Raven

 Activating environment at `~/REPOS/Raven.jl/Project.toml`


In [2]:
edges=Raven.readToFeTedges(open("../testdata/10chain.edge","r"))

10×10 SparseArrays.SparseMatrixCSC{Float64,Int64} with 18 stored entries:
  [2 ,  1]  =  0.1
  [1 ,  2]  =  0.1
  [3 ,  2]  =  0.1
  [2 ,  3]  =  0.1
  [4 ,  3]  =  0.1
  [3 ,  4]  =  0.1
  [5 ,  4]  =  0.1
  [4 ,  5]  =  0.1
  [6 ,  5]  =  0.1
  [5 ,  6]  =  0.1
  [7 ,  6]  =  0.1
  [6 ,  7]  =  0.1
  [8 ,  7]  =  0.1
  [7 ,  8]  =  0.1
  [9 ,  8]  =  0.1
  [8 ,  9]  =  0.1
  [10,  9]  =  0.1
  [9 , 10]  =  0.1

In [3]:
import UnicodePlots
UnicodePlots.spy(edges)


      Sparsity Pattern
      ┌─────┐    
    1 │⢔⢄⠀⠀⠀│ > 0
      │⠀⠑⢕⢄⠀│ < 0
   10 │⠀⠀⠀⠑⠕│    
      └─────┘    
      1    10
      nz = 18

In [4]:
rates=Raven.ratematrix(edges,Raven.rateAdiabatic)

10×10 SparseArrays.SparseMatrixCSC{Float64,Int64} with 28 stored entries:
  [1 ,  1]  =  -9.54587e13
  [2 ,  1]  =  9.54587e13
  [1 ,  2]  =  9.54587e13
  [2 ,  2]  =  -1.90917e14
  [3 ,  2]  =  9.54587e13
  [2 ,  3]  =  9.54587e13
  [3 ,  3]  =  -1.90917e14
  [4 ,  3]  =  9.54587e13
  [3 ,  4]  =  9.54587e13
  [4 ,  4]  =  -1.90917e14
  [5 ,  4]  =  9.54587e13
  [4 ,  5]  =  9.54587e13
  ⋮
  [6 ,  6]  =  -1.90917e14
  [7 ,  6]  =  9.54587e13
  [6 ,  7]  =  9.54587e13
  [7 ,  7]  =  -1.90917e14
  [8 ,  7]  =  9.54587e13
  [7 ,  8]  =  9.54587e13
  [8 ,  8]  =  -1.90917e14
  [9 ,  8]  =  9.54587e13
  [8 ,  9]  =  9.54587e13
  [9 ,  9]  =  -1.90917e14
  [10,  9]  =  9.54587e13
  [9 , 10]  =  9.54587e13
  [10, 10]  =  -9.54587e13

In [5]:
# OK, let's look at the SVD (dense) method, as a reference
SVD=Raven.solveMasterDense(rates)

LinearAlgebra.SVD{Float64,Float64,Array{Float64,2}}
U factor:
10×10 Array{Float64,2}:
 -0.0699596  -0.138197  -0.203031   …   0.425325      0.441708   0.316228
  0.203031    0.361803   0.441708       0.262866      0.39847    0.316228
 -0.316228   -0.447214  -0.316228      -1.11022e-16   0.316228   0.316228
  0.39847     0.361803  -0.0699596     -0.262866      0.203031   0.316228
 -0.441708   -0.138197   0.39847       -0.425325      0.0699596  0.316228
  0.441708   -0.138197  -0.39847    …  -0.425325     -0.0699596  0.316228
 -0.39847     0.361803   0.0699596     -0.262866     -0.203031   0.316228
  0.316228   -0.447214   0.316228      -3.17888e-16  -0.316228   0.316228
 -0.203031    0.361803  -0.441708       0.262866     -0.39847    0.316228
  0.0699596  -0.138197   0.203031       0.425325     -0.441708   0.316228
singular values:
10-element Array{Float64,1}:
 3.724905314918553e14
 3.4537272297579325e14
 3.031357459826044e14
 2.4991405010270456e14
 1.909173457461779e14
 1.3192064138965

In [6]:
# Fiddle till we extract the value we want: the very small singular values, and their right vectors Vt
ρ=SVD.Vt[10,:]

10-element Array{Float64,1}:
 -0.3162277660168386
 -0.31622776601683855
 -0.316227766016838
 -0.31622776601683744
 -0.3162277660168371
 -0.31622776601683744
 -0.3162277660168376
 -0.31622776601683755
 -0.3162277660168379
 -0.31622776601683805

In [7]:
sum(abs.(ρ))

3.162277660168378

In [8]:
using IterativeSolvers

In [108]:
siteE=randn(10)./1
rates=Raven.ratematrix(edges,siteE,Raven.rateMarcus)/1e13
#rates=Raven.ratematrix(edges,Raven.rateAdiabatic)/1E13
rates

10×10 SparseArrays.SparseMatrixCSC{Float64,Int64} with 28 stored entries:
  [1 ,  1]  =  -0.063536
  [2 ,  1]  =  23.631
  [1 ,  2]  =  0.063536
  [2 ,  2]  =  -29.2449
  [3 ,  2]  =  4.58574
  [2 ,  3]  =  5.61386
  [3 ,  3]  =  -27.3396
  [4 ,  3]  =  0.0178524
  [3 ,  4]  =  22.7538
  [4 ,  4]  =  -18.9605
  [5 ,  4]  =  0.00384622
  [4 ,  5]  =  18.9427
  ⋮
  [6 ,  6]  =  -12.8964
  [7 ,  6]  =  1.32248
  [6 ,  7]  =  6.12213e-7
  [7 ,  7]  =  -1.32248
  [8 ,  7]  =  2.67453e-30
  [7 ,  8]  =  8.83518e-15
  [8 ,  8]  =  -0.00347557
  [9 ,  8]  =  18.6334
  [8 ,  9]  =  0.00347557
  [9 ,  9]  =  -18.6334
  [10,  9]  =  0.129287
  [9 , 10]  =  3.29896e-9
  [10, 10]  =  -0.129287

In [109]:
exp(collect(-rates))

10×10 Array{Float64,2}:
      2.795e11          -3.95384e11   …        1.04172e-20  -1.02884e-30
     -1.47056e14         2.08027e14           -5.48552e-18   5.41951e-28
      1.10087e14        -1.55735e14            4.1264e-18   -4.08451e-28
     -1.35244e11         1.9133e11            -5.63669e-21   5.93351e-31
      1.55322e7         -2.19739e7             3.88951e-23  -6.85394e-33
     -9.72323e6          1.37568e7    …       -1.25114e-18   2.22313e-28
 399624.0          -565439.0                   1.18127e-11  -2.10559e-21
     -3.19096e-26        4.51538e-26      -23149.4           4.12636e-6
      3.9939e-26        -5.65462e-26           1.2411e8     -0.0221224
     -1.54587e-28        2.18939e-28     -866981.0           1.13817

In [110]:
siteE

10-element Array{Float64,1}:
 -0.049632867575019404
  0.4310635766397972
  0.41463455320756937
 -0.16609144163282313
 -0.8566001677950934
 -0.04236498835138833
  1.1422333316571767
 -1.759934596555404
 -1.062532953662289
  0.3574503007891717

In [111]:
rates

10×10 SparseArrays.SparseMatrixCSC{Float64,Int64} with 28 stored entries:
  [1 ,  1]  =  -0.063536
  [2 ,  1]  =  23.631
  [1 ,  2]  =  0.063536
  [2 ,  2]  =  -29.2449
  [3 ,  2]  =  4.58574
  [2 ,  3]  =  5.61386
  [3 ,  3]  =  -27.3396
  [4 ,  3]  =  0.0178524
  [3 ,  4]  =  22.7538
  [4 ,  4]  =  -18.9605
  [5 ,  4]  =  0.00384622
  [4 ,  5]  =  18.9427
  ⋮
  [6 ,  6]  =  -12.8964
  [7 ,  6]  =  1.32248
  [6 ,  7]  =  6.12213e-7
  [7 ,  7]  =  -1.32248
  [8 ,  7]  =  2.67453e-30
  [7 ,  8]  =  8.83518e-15
  [8 ,  8]  =  -0.00347557
  [9 ,  8]  =  18.6334
  [8 ,  9]  =  0.00347557
  [9 ,  9]  =  -18.6334
  [10,  9]  =  0.129287
  [9 , 10]  =  3.29896e-9
  [10, 10]  =  -0.129287

In [112]:
r=IterativeSolvers.lobpcg(rates,  true, 1)

LinearAlgebra.PosDefException: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [107]:
r.X/sum(r.X)

10×1 Array{Float64,2}:
  0.23510382540272923
  0.021724810814724757
 -0.06824274739080707
  1.0708045455879314
 -0.8602747543496092
  0.40214387313832717
  0.337654353092406
 -0.7853785232395075
  0.6735118665612476
 -0.02704724961744243

In [98]:
# check stationary condition
exp(collect(rates)) * r.X

10×1 Array{Float64,2}:
 9.382582098782643e-25
 1.8912998217590035e-24
 3.890035129831653e-24
 6.731742653549812e-24
 9.375966908443555e-24
 1.1706606049401429e-23
 1.4352131760717098e-23
 1.6781485144257016e-23
 1.853744607891064e-23
 1.9608413540623544e-23

In [69]:
# Chebyshev method, which enables you to declare the region of eigenvalue interest
IterativeSolvers.chebyshev(rates,zeros(rates.m),0.0,1)

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [78]:
# Power method approach to getting the dominant eigenvalue
λ,x=IterativeSolvers.powm(rates,shift=0) #, shift=0.0)

@show λ
x

λ = -34.95339790830564 - 5.551115123125783e-17im


10-element Array{Complex{Float64},1}:
  0.05040689912504277 + 0.13946036732792688im
 -0.11388758072823192 - 0.3724398110980073im
   0.1056936497413206 + 0.48474643237552567im
 -0.06609820100278006 - 0.44563174414292084im
  0.06190747030406556 + 0.2775403707334943im
 -0.11571008296375589 - 0.04671233894994546im
  0.18802821961466487 - 0.16020808357474012im
 -0.21856803006745967 + 0.2671661661696742im
  0.17460683468395108 - 0.2383455001185307im
 -0.06637917870681732 + 0.09442414127752329im

In [73]:
# partial SVD factorisation using the pseudo-inverse
Σ,L=IterativeSolvers.svdl(rates, nsv=1, vecs=:right)
@show Σ.S
@show Σ.Vt
L

Σ.S = [37.23745620468033]
Σ.Vt = [0.07479892281886896 -0.21985565528051323 0.33180144319311183 -0.4175721111828822 0.44263771431722954 -0.4393647928098618 0.38066236193913094 -0.2966388352969385 0.19015299814860084 -0.059864712798223224]


IterativeSolvers.PartialFactorization{Float64,Float64}([-0.07777663511711061 0.1109756025003652; 0.22399277958248492 -0.34858820073339214; … ; -0.1807968643956677 -0.40176247435746615; 0.06179633809897178 0.1141252769457311], [0.07914328029742114 -0.1576862451821319 -0.0029734595172833107; -0.22425545307605385 0.15772666810706673 0.35533141373452715; … ; 0.17724998138899975 0.4739801558342976 0.011738124906875277; -0.06381124063630692 0.14335693375969832 -0.4235971622043112], [37.23745620468033 0.4213661743419432; 0.0 27.426408918524483], 20.907509978587186)

In [74]:
Raven.solveMasterCG(rates)

10-element SparseArrays.SparseVector{Float64,Int64} with 0 stored entries

In [ ]:
import DelimitedFiles
raw=DelimitedFiles.readdlm(open("../testdata/10chain.xyz","r"))

In [ ]:
XYZ=raw[:,1:3]

In [ ]:
xyz=Raven.readToFeTxyz(open("../testdata/10chain.xyz","r"))

In [ ]:
"Single iteration of Yu's power iterative method to solve master equation. Needs a bare ratematrix."
function YuPRBpowm!(ω, P)
    for i in eachindex(P)
        P[i] = sum(ω[i,:] .* P[:]) / sum(ω[:,i]) / (1 - sum( (ω[:,i].-ω[i,:]) .* P[:])/sum(ω[:,i]) ) 
    end
    P
end

In [ ]:
rates=Raven.bareratematrix(edges,Raven.rateMarcus)

In [ ]:
P=vcat(ones(1),zeros(9))

for i in 1:50
    Pold=copy(P)
    Raven.YuPRBpowm!(rates,P)
    P=0.6*P + 0.4*Pold
    @show P
end
sum(P)

In [ ]:

for i in 1:100
    Raven.YuPRBpowm!(rates,P)
    @show P
end


In [ ]:
ω=rates/sum(rates)
i=1
ω[i,:] .* P[:]

In [ ]:
1 - sum( (ω[:,i].-ω[i,:]) .* P[:])

In [ ]:
rates[2,:]

In [ ]:
P=vcat(zeros(8),ones(2))
c=0
while true
    c=c+1
    Pold=copy(P)
    Raven.YuPRBpowm!(rates,P)
    
    @show P
    if sum(abs.(P-Pold))<0.0001 
        break
    end
end
return c



In [ ]:
function YuPRBpowm_converge!(rates,P; eps=0.0001)
    c=0
    while true
        c=c+1
        Pold=copy(P)
        Raven.YuPRBpowm!(rates,P)
    
        @show P
        if sum(abs.(P-Pold))<eps
            break
        end
    end
    return c
end

In [ ]:
P=vcat(zeros(8),ones(2))
@time YuPRBpowm_converge!(rates,P)

In [ ]:
@time YuPRBpowm!(rates,P)

In [ ]:
P=vcat(zeros(8),ones(2))
@time YuPRBpowm_converge!(rates,P)

In [ ]:
@code_warntype YuPRBpowm_converge!(rates,P)

In [ ]:
@code_warntype YuPRBpowm!(rates,P)